In [2]:
%pip install torch
import torch
if not torch.cuda.is_available():
  %pip install tensorflow==2.14 intel-extension-for-tensorflow[xpu]==2.14 intel-optimization-for-horovod==0.28.1.1 torch==2.0.1 torchvision==0.15.2 intel_extension_for_pytorch==2.0.100 oneccl-bind-pt==2.0.0 --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/ -q

In [3]:
%pip install wandb numpy transformers datasets transformers[torch] accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, whi

In [32]:
use_wandb = False

if use_wandb:
    import wandb

    wandb.init(project="codeBERTa-on-csharp")

In [6]:
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
)


model_name = "huggingface/CodeBERTa-small-v1"

# download prepare the data
# dataset = load_dataset("code_search_net","javascript")
dataset = load_dataset("microsoft/LCC_csharp")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/994k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/483k [00:00<?, ?B/s]

In [27]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(300))
small_train_dataset = small_train_dataset.map(
    lambda x: tokenizer(x["context"], truncation=True, padding="max_length"),
    batched=True,
    remove_columns=dataset["train"].column_names,
)
train_labels = small_train_dataset['input_ids']
small_train_dataset.add_column(name='labels', column=train_labels)


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 300
})

In [28]:

small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))
small_eval_dataset = small_eval_dataset.map(
    lambda x: tokenizer(x["context"], truncation=True, padding="max_length"),
    batched=True,
    remove_columns=dataset["test"].column_names,
)
eval_labels = small_eval_dataset['input_ids']
small_eval_dataset.add_column(name='labels', column=eval_labels)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [9]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [30]:
small_eval_dataset.features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [34]:
import os


model = AutoModelForMaskedLM.from_pretrained(model_name)


if use_wandb:

    os.environ["WANDB_PROJECT"] = "codeBERTa-on-csharp"

    os.environ["WANDB_LOG_MODEL"] = "true"

    os.environ["WANDB_WATCH"] = "false"



# pass "wandb" to the 'report_to' parameter to turn on wandb logging



training_args = TrainingArguments(
    output_dir="models",

    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,

    weight_decay=0.01,
    # push_to_hub=True,

)
if use_wandb:
    training_args.report_to = ["wandb"]

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.745043
2,No log,1.636869
3,No log,1.698277


TrainOutput(global_step=114, training_loss=1.653344003777755, metrics={'train_runtime': 131.0616, 'train_samples_per_second': 6.867, 'train_steps_per_second': 0.87, 'total_flos': 119364422860800.0, 'train_loss': 1.653344003777755, 'epoch': 3.0})

In [36]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

if use_wandb:
    wandb.finish()

Perplexity: 5.64


In [42]:
%pip uninstall -y numpy torch
%pip install transformers
from transformers import pipeline

text = """
//
// Author:
//   Jb Evain (jbevain@gmail.com)
//
// Copyright (c) 2008 - 2015 Jb Evain
// Copyright (c) 2008 - 2011 Novell, Inc.
//
// Licensed under the MIT/X11 license.
//

namespace Mono.Cecil.Cil {

	public sealed class VariableDefinition : VariableReference {

		public bool IsPinned {
			get { return variable_type.IsPinned; }
		}

		public VariableDefinition (TypeReference variableType)
			: base (variableType)
		{
		}

		public override VariableDefinition Resolve ()
		{
			return this;
		}
	}
}"""

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)
pipe(text)

Found existing installation: numpy 1.26.2
Uninstalling numpy-1.26.2:
  Successfully uninstalled numpy-1.26.2
Found existing installation: torch 2.1.0+cu118
Uninstalling torch-2.1.0+cu118:
  Successfully uninstalled torch-2.1.0+cu118
  Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.24.1 requires torch>=1.10.0, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
torchtext 0.16.0 requires torch==2.1.0, which is not installed.
torchvision 0.16.0+cu118 requires torch==2.1.0, which is not installed.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1

RuntimeError: ignored